In [1]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
from pyproj import Transformer
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time

In [ ]:
# Construct Supply Dataset
# Columns needed: Supply Site, Technology Type, Setup Cost, Capacity, Longitude, Latitude
# Supply site I have, Technology Type I have
# Setup Cost I have to infer using Capacity and energy capital costs
# Capacity I have, but its in MW hours and I need to convert to GW per year
# This requires also the use of capacity factors as these are used to get an upper bound
# on how much energy can be delivered
# Longitude and Latitude I need to obtain from  X and Y coordinates in dataset
# Dataset is repd_q3_oct-2024.csv
# Oh also I want to ignore those technologies that I don't have cap costs for


complete_supply = pd.read_csv("Data/repd-q3-oct-2024.csv", encoding='latin1')
# drop empty values
complete_supply['Installed Capacity (MWelec)'] = complete_supply['Installed Capacity (MWelec)'].str.strip()  # Remove extra spaces
complete_supply = complete_supply[complete_supply['Installed Capacity (MWelec)'] != '']  # Drop rows with empty values

complete_supply['Installed Capacity (MWelec)'] = pd.to_numeric(complete_supply['Installed Capacity (MWelec)'])


# Create Setup Cost column
# These are build costs in USD per kW capacity from US Annual Energy Outlook 2022
energy_capital_costs = {
    "Wind Onshore": 1462,
    "Wind Offshore": 3285,
    "Solar Photovoltaics": 1327,
    "Large Hydro": 8653,
    "Small Hydro": 2574,
}
complete_supply = complete_supply.loc[complete_supply['Technology Type'].isin(energy_capital_costs.keys())]
complete_supply['Capital Costs per kW'] = complete_supply['Technology Type'].map(energy_capital_costs)
complete_supply['Setup Costs'] = complete_supply['Installed Capacity (MWelec)'] * complete_supply['Capital Costs per kW'] * 1000

# Create Annual Capacity (GW) - GW will is a sensible magnitude to choose
# Max Annual Capacity (GWh) = (Capacity (MW) * hours_in_year * capacity_factor)/1000
capacity_factors = {
    "Wind Offshore": 0.3954,
    "Wind Onshore": 0.2464,
    "Solar Photovoltaics": 0.1015,
    "Large Hydro": 37.5,
    "Small Hydro": 32.3,
}
complete_supply['Capacity Factor'] = complete_supply['Technology Type'].map(capacity_factors)
hours_in_year = 24 * 365
complete_supply['Annual Capacity (GW)'] = (
    complete_supply['Installed Capacity (MWelec)'] * hours_in_year * complete_supply['Capacity Factor'] / 1000
)

# Create Longitude and Latitude coordinates from national grid coordinates given
transformer = Transformer.from_crs("epsg:27700", "epsg:4326", always_xy=True)
complete_supply[['Longitude', "Latitude"]] = complete_supply.apply(
    lambda row: pd.Series(transformer.transform(row["X-coordinate"], row["Y-coordinate"])),
    axis=1
)

# Remove columns with invalid longitude/latitude coords
invalid_supply_rows = complete_supply[
    (complete_supply['Latitude'].isna()) | (complete_supply['Longitude'].isna()) 
    | (~complete_supply['Latitude'].between(-90, 90)) | (~complete_supply['Longitude'].between(-180, 180))
]
complete_supply = complete_supply.drop(invalid_supply_rows.index)

complete_supply = complete_supply.rename(columns={
    "Site Name": "Supply Site"
})
complete_supply = complete_supply.drop_duplicates(subset=['Supply Site'], keep='first')


supply_dataset_cols = ['Supply Site', 'Technology Type', 'Setup Costs',
                       'Annual Capacity (GW)', 'Longitude', 'Latitude']
complete_supply = complete_supply[supply_dataset_cols]
complete_supply.to_csv("Data/complete_supply.csv")

In [ ]:
# Create demand dataset
# This will contain Demand Site, Annual Demand (GWh), Year, Latitude, Longitude
# Demand Site I have, Annual Demand I have, Year I have 
# Latitude and Longitude I need to calculate from the local authority name

xls = pd.ExcelFile('Data/Subnational_electricity_consumption_statistics_2005-2022.xlsx')
sheet_names = [sheet for sheet in xls.sheet_names if sheet != "Cover sheet"]
demand_datasets = []
excluded_vals = ['All local authorities']
for sheet in sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet, skiprows=4)
    df = df.rename(columns={
        'Local authority': 'Demand Site',
        'Total consumption\n(GWh):\nAll meters': "Annual Demand (GWh)"
    })
    df['Year'] = sheet
    df = df[['Demand Site', 'Annual Demand (GWh)', 'Year']]
    df = df.loc[~df['Demand Site'].isin(excluded_vals)]
    demand_datasets.append(df)
demand_data = pd.concat(demand_datasets, ignore_index=True)

# I don't want to consider local authorities that are not in every year
years_per_authority = demand_data.groupby('Demand Site')['Year'].nunique()
total_years = demand_data['Year'].nunique()
authorities_in_all_years = years_per_authority[years_per_authority == total_years].index
demand_data = demand_data[demand_data['Demand Site'].isin(authorities_in_all_years)]

geolocator = Nominatim(user_agent="geoapi")

local_authorities = list(demand_data['Demand Site'].unique())

def get_coordinates(location_name):
    try:
        location = geolocator.geocode(f"{location_name}, UK")
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error for {location_name} :{e}")
        return None, None
coords = {}
for authority in local_authorities:
    coords[authority] = get_coordinates(authority)
    time.sleep(0.01)
# coords

{'Darlington': (54.5242081, -1.5555812),
 'Gateshead': (54.9625789, -1.6019294),
 'Hartlepool': (54.6857276, -1.2093696),
 'Middlesbrough': (54.5760419, -1.2344047),
 'Newcastle upon Tyne': (54.9738474, -1.6131572),
 'North Tyneside': (55.02979945, -1.5082559996141889),
 'Redcar and Cleveland': (54.5679056, -1.0054963165760897),
 'South Tyneside': (54.969874250000004, -1.4476805465645368),
 'Stockton-on-Tees': (54.564094, -1.3129164),
 'Sunderland': (54.9058512, -1.3828727),
 'Blackburn with Darwen': (53.699176949999995, -2.4709000953138327),
 'Blackpool': (53.8179442, -3.0509812),
 'Bolton': (53.5782863, -2.4300367),
 'Burnley': (53.7907262, -2.2439196),
 'Bury': (52.2460367, 0.7125173),
 'Chorley': (53.6531915, -2.6294313),
 'Fylde': (53.793356, -2.894251009596323),
 'Halton': (53.35385255, -2.7427828785393507),
 'Hyndburn': (53.7607317, -2.39005491388217),
 'Lancaster': (54.0484068, -2.7990345),
 'Liverpool': (53.4071991, -2.99168),
 'Manchester': (53.4794892, -2.2451148),
 'Pendle'

In [7]:
latitude_dict = {place: cord[0] for place, cord in coords.items()}
longitude_dict = {place: cord[1] for place, cord in coords.items()}
demand_data['Latitude'] = demand_data['Demand Site'].map(latitude_dict)
demand_data['Longitude'] = demand_data['Demand Site'].map(longitude_dict)
demand_data.to_csv("Data/complete_demand.csv")

In [18]:
# Derive the cost DF.
# Need to calculate the costs for supplying a GW of energy to each location from each supply site. 
# This dataset needs Demand Site, Supply Site, Cost - for now I will assume a time independent supply cost
# If I want to expand the space I can consider time varying supply costs also, woudl be more accurate for how this woudl be considered

# Generation costs $ per MWh from IRENA 2020
energy_generation_costs = {
    "Wind Offshore": 115,
    "Wind Onshore": 59,
    "Solar Photovoltaics": 68,
    "Large Hydro": 47,
    "Small Hydro": 47,
}

# Transmission costs per km per MW
energy_transmission_cost = 15000

#Remove duplicates in demand because we don't consider time dependence (yet)
unique_demand = demand_data.drop_duplicates(subset=['Demand Site'], keep='first')

# Take Cartesian Proudct of Supply and Demand sites to get initial cost pairs
cost_df = complete_supply[['Supply Site']].merge(
    unique_demand[['Demand Site']], how='cross')

# Add the technology types
cost_df = cost_df.merge(complete_supply[['Supply Site', 'Technology Type']],
                        on='Supply Site', how='left')

# Add supply and demand longitude and latitude coords
cost_df = cost_df.merge(
    complete_supply[['Supply Site', 'Latitude', 'Longitude']],on='Supply Site', how='left').rename(
        columns={'Latitude': "Supply Latitude", 'Longitude': 'Supply Longitude'})
cost_df = cost_df.merge(
    unique_demand[['Demand Site', 'Latitude', 'Longitude']], on='Demand Site', how='left').rename(
        columns={'Latitude': 'Demand Latitude', 'Longitude': 'Demand Longitude'})

# apply takes too long - vectorise the approach instead
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)* np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371 # Earth's radius
    return c * r

cost_df['Distance (km)'] = haversine(
    cost_df['Supply Latitude'], cost_df['Supply Longitude'], cost_df['Demand Latitude'], cost_df['Demand Longitude']
)
# Calculate transmission cost ($ per GWh)
cost_df['Transmission Cost ($ per GWh)'] = cost_df['Distance (km)'] * energy_transmission_cost * 1000

# Add generation costs ($ per MWh)
cost_df['Generation Cost ($ per GWh)'] = cost_df['Technology Type'].map(energy_generation_costs) * 1000

cost_df['Supply Cost'] = cost_df['Transmission Cost ($ per GWh)'] + cost_df['Generation Cost ($ per GWh)']
years = pd.DataFrame({'Year':demand_data['Year'].unique()})
cost_df = cost_df.merge(years, how='cross')
cost_df.to_csv("Data/complete_cost.csv")
    

In [24]:
# Now I will compile smaller datasets to gradually test on. 
# Use the largest supply plants and demand plants gradually moving down the list

def create_subsets(demand_df, supply_df, costs_df, n_supply, n_demand):
    
    # Get largest demand sites
    top_demand_sites = (
        demand_df.groupby('Demand Site')['Annual Demand (GWh)']
        .sum().nlargest(n_demand).index
    )
    demand_subset = demand_df[demand_df['Demand Site'].isin(top_demand_sites)]

    # Get largest supply sites
    supply_subset = supply_df.nlargest(n_supply, 'Annual Capacity (GW)')

    costs_subset = costs_df[costs_df['Supply Site'].isin(supply_subset['Supply Site']) &
                            costs_df['Demand Site'].isin(demand_subset['Demand Site'])]
    
    return demand_subset, supply_subset, costs_subset



In [ ]:
from utils.feasibility_check import check_feasibility

# Test 1
demand_n3, supply_n5, cost_n3_n5 = create_subsets(demand_data, complete_supply, cost_df, n_demand=3, n_supply=5)
feasible = check_feasibility(demand_n3, supply_n5).iloc[0]
print(feasible)
if feasible:
    demand_n3.to_csv("Data/test_1_demand_n3.csv", index=False)
    supply_n5.to_csv("Data/test_1_supply_n5.csv", index=False)
    cost_n3_n5.to_csv("Data/test_1_costs_n3_n5.csv", index=False)

# Test 2
demand_n10, supply_n10, costs_n10_n10 = create_subsets(demand_data, complete_supply, cost_df, n_demand=10, n_supply=10)
if check_feasibility(demand_n10, supply_n10).iloc[0]:
    demand_n10.to_csv("Data/test_2_demand_n10.csv", index=False)
    supply_n10.to_csv("Data/test_2_supply_n10.csv", index=False)
    costs_n10_n10.to_csv("Data/test_2_costs_n10_n10.csv", index=False)

# Test 3
demand_n30, supply_n30, costs_n30_n30 = create_subsets(demand_data, complete_supply, cost_df, n_demand=30, n_supply=30)
if check_feasibility(demand_n10, supply_n10).iloc[0]:
    demand_n30.to_csv("Data/test_3_demand_n30.csv", index=False)
    supply_n30.to_csv("Data/test_3_supply_n30.csv", index=False)
    costs_n30_n30.to_csv("Data/test_3_costs_n30_n30.csv", index=False)

# Test 4
demand_n50, supply_n50, costs_n50_n50 = create_subsets(demand_data, complete_supply, cost_df, n_demand=50, n_supply=50)
if check_feasibility(demand_n10, supply_n10).iloc[0]:
    demand_n50.to_csv("Data/test_4_demand_n50.csv", index=False)
    supply_n50.to_csv("Data/test_4_supply_n50.csv", index=False)
    costs_n50_n50.to_csv("Data/test_4_costs_n50_n50.csv", index=False)


# Test 5
demand_n100, supply_n100, costs_n100_n100 = create_subsets(demand_data, complete_supply, cost_df, n_demand=100, n_supply=100)
if check_feasibility(demand_n10, supply_n10).iloc[0]:
    demand_n100.to_csv("Data/test_5_demand_n100.csv", index=False)
    supply_n100.to_csv("Data/test_5_supply_n100.csv", index=False)
    costs_n100_n100.to_csv("Data/test_5_costs_n100_n100.csv", index=False)



True


In [68]:
from utils.feasibility_check import check_feasibility
# print(check_feasibility(demand_n3, supply_n5))
max_demand = demand_n3[['Year', 'Annual Demand (GWh)']].groupby('Year').sum().max().iloc[0]
total_supply = supply_n5['Annual Capacity (GW)'].sum()
print(max_demand, total_supply, max_demand < total_supply)
# demand_n3.to_csv("Data/demand_n3.csv", index=False)
# supply_n5.to_csv("Data/supply_n5.csv", index=False)
# cost_n3_n5.to_csv("Data/costs_n3_n5.csv", index=False)

12511.96353005 93369.6864 True


In [51]:
baseline_output = pd.read_csv("Outputs/supply_results_baseline_test_1.csv")
lr_output = pd.read_csv("Outputs/supply_results_LR_test_1.csv")
baseline_output.equals(lr_output)

True

True